In [ ]:
from data.molfamily import get_family_groups
from data.graphdataset import ThermoMLDataset
import polars as pl

test_dt = ThermoMLDataset("./data/thermoml")
families = {"inchis": [], "famgroup": []}
for graph in test_dt:
    inchi = graph.InChI
    famgroup = get_family_groups(inchi)
    if len(famgroup) > 1:
        famgroup = ["polyfunctional"]
    families["inchis"].append(inchi)
    families["famgroup"].append(famgroup[0])
families = pl.DataFrame(families)
families.write_csv('families.csv')